# **📓 01: Preprocesamiento y Organización del Dataset**

Este notebook es el corazón del pipeline de gestión de datos. Su propósito es tomar los datos crudos y etiquetados y organizarlos en una estructura de directorios lista para el entrenamiento.

**Tareas Principales:**
1.  **Sincronización:** Lee el archivo `.csv` de etiquetas y lo compara con el registro maestro (`dataset_map.csv`) para identificar archivos nuevos y cambios en las etiquetas.
2.  **Separación por Fuente:** Procesa los datos de las fuentes `propia` y `externa` de manera independiente.
3.  **División Estratificada:** Divide los nuevos archivos en conjuntos de `train`, `validation` y `test`, asegurando que la proporción de clases se mantenga.
4.  **Organización de Archivos:** Mueve físicamente los archivos de imagen desde las carpetas `fotos_crudas` a la estructura final del dataset.
5.  **Actualización de Registro:** Guarda el estado final en `dataset_map.csv` para mantener la trazabilidad.

In [ ]:
# ====================================================================================
# @title PASO 1: SETUP DEL ENTORNO
# ====================================================================================
import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split
from google.colab import drive

print("✅ Entorno listo.")

### **Configuración del Entorno y Búsqueda de Rutas**
Esta celda se encarga de dos tareas críticas:
1.  **Montar Google Drive.**
2.  **Localizar dinámicamente la carpeta del proyecto** en Drive, asegurando que el notebook funcione para cualquier colaborador.
3.  **Importar la configuración** desde un archivo `config.py`. Este archivo contiene las rutas y IDs específicos del entorno de cada usuario, separando la configuración del código.

In [ ]:
# ====================================================================================
# @title PASO 2: CONFIGURACIÓN DE RUTAS Y CONEXIÓN
# ====================================================================================

# Montar Google Drive
if not os.path.exists('/content/drive/MyDrive'):
    drive.mount('/content/drive')
else:
    print("Google Drive ya está montado.")

# --- Búsqueda Dinámica de la Ruta Base del Proyecto ---
NOMBRE_CARPETA_ANCLA = 'Proyecto_VIREC'
RUTA_BASE_PROYECTO = None
print(f"\nBuscando la carpeta ancla '{NOMBRE_CARPETA_ANCLA}'...")
for root, dirs, files in os.walk('/content/drive/MyDrive'):
    if NOMBRE_CARPETA_ANCLA in dirs:
        RUTA_BASE_PROYECTO = os.path.join(root, NOMBRE_CARPETA_ANCLA)
        break

if not RUTA_BASE_PROYECTO:
    raise FileNotFoundError(f"❌ ERROR CRÍTICO: No se pudo encontrar la carpeta '{NOMBRE_CARPETA_ANCLA}'.")
else:
    print(f"✅ ¡Proyecto encontrado! La ruta base es: {RUTA_BASE_PROYECTO}")
    
    # --- Importar la configuración del usuario desde config.py ---
    # Esto permite que cada usuario defina sus propias rutas sin modificar el código.
    import sys
    sys.path.append(RUTA_BASE_PROYECTO)
    try:
        from config import ID_CARPETA_PROPIAS, ID_CARPETA_EXTERNAS
        print("✅ Configuración local importada desde config.py.")
    except ImportError:
        raise ImportError("❌ ERROR: No se encontró el archivo 'config.py'. Por favor, crea una copia de 'config.py.template', renómbrala y rellena tus IDs de carpeta.")

    # --- Construcción de todas las rutas del proyecto ---
    RUTA_DATASET = os.path.join(RUTA_BASE_PROYECTO, 'dataset')
    RUTA_FOTOS_CRUDAS_PROPIAS = os.path.join(RUTA_DATASET, 'fotos_crudas_propias')
    RUTA_FOTOS_CRUDAS_EXTERNAS = os.path.join(RUTA_DATASET, 'fotos_crudas_externas')
    RUTA_DATASET_FINAL_PROPIO = os.path.join(RUTA_DATASET, 'dataset_final_propio')
    RUTA_DATASET_FINAL_EXTERNO = os.path.join(RUTA_DATASET, 'dataset_final_externo')
    RUTA_CSV_ETIQUETAS = os.path.join(RUTA_BASE_PROYECTO, 'VIREC - Hoja de Etiquetas del Dataset.csv')
    RUTA_REGISTRO = os.path.join(RUTA_BASE_PROYECTO, 'dataset_map.csv')
    
    print("✅ Rutas configuradas exitosamente.")

### **Ejecución del Pipeline de Organización**
Esta celda contiene la lógica principal del notebook. Ejecútala para sincronizar el CSV de etiquetas con las carpetas de datos y organizar los archivos en la estructura `train/validation/test`.

In [ ]:
# ====================================================================================
# @title PASO 3: EJECUTAR PIPELINE DE PREPROCESAMIENTO
# ====================================================================================

def procesar_fuente(df_deseado_fuente, df_actual_completo, ruta_fotos_crudas, ruta_dataset_final, nombre_fuente):
    """
    Procesa un subconjunto de datos (propio o externo), maneja cambios de etiquetas,
    divide los nuevos archivos y los mueve a su destino final.
    """
    print("\n" + "="*60)
    print(f"            PROCESANDO FUENTE: {nombre_fuente.upper()}")
    print("="*60)
    
    df_actual_fuente = df_actual_completo[df_actual_completo['fuente'] == nombre_fuente]
    archivos_procesados = set(df_actual_fuente['nombre_archivo'])
    df_nuevos = df_deseado_fuente[~df_deseado_fuente['nombre_archivo'].isin(archivos_procesados)]
    
    # 1. Identificar y procesar etiquetas cambiadas
    df_merged = pd.merge(df_deseado_fuente, df_actual_fuente, on='nombre_archivo', suffixes=('_deseado', '_actual'), how='inner')
    df_cambiados = df_merged[df_merged['etiqueta_deseado'] != df_merged['etiqueta_actual']]

    if not df_cambiados.empty:
        print(f"Se detectaron {len(df_cambiados)} archivos con etiquetas cambiadas. Corrigiendo...")
        for _, row in df_cambiados.iterrows():
            archivo, etiqueta_vieja, etiqueta_nueva, split = row['nombre_archivo'], row['etiqueta_actual'], row['etiqueta_deseado'], row['split']
            if pd.notna(split):
                ruta_vieja = os.path.join(ruta_dataset_final, split, etiqueta_vieja, archivo)
                ruta_nueva = os.path.join(ruta_dataset_final, split, etiqueta_nueva, archivo)
                if os.path.exists(ruta_vieja):
                    shutil.move(ruta_vieja, ruta_nueva)
                    df_actual_completo.loc[df_actual_completo['nombre_archivo'] == archivo, 'etiqueta'] = etiqueta_nueva
                    print(f"  - '{archivo}' movido de '{etiqueta_vieja}' a '{etiqueta_nueva}'.")
    else:
        print("No se detectaron cambios en las etiquetas de archivos existentes.")

    # 2. Procesar archivos nuevos
    if df_nuevos.empty:
        print("No hay archivos nuevos para añadir en esta fuente.")
        return pd.DataFrame()

    print(f"\nSe encontraron {len(df_nuevos)} archivos nuevos para procesar.")
    
    # Lógica de división robusta
    train_df, validation_df, test_df = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    if len(df_nuevos) < 5:
        train_df = df_nuevos.copy()
    else:
        if df_nuevos['etiqueta'].nunique() < 2:
            train_df, temp_df = train_test_split(df_nuevos, test_size=0.2, random_state=42)
        else:
            train_df, temp_df = train_test_split(df_nuevos, test_size=0.2, random_state=42, stratify=df_nuevos['etiqueta'])
        if temp_df['etiqueta'].nunique() < 2 or temp_df['etiqueta'].value_counts().min() < 2:
            validation_df = temp_df.copy()
        else:
            validation_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['etiqueta'])

    if not train_df.empty: train_df.loc[:, 'split'] = 'train'
    if not validation_df.empty: validation_df.loc[:, 'split'] = 'validation'
    if not test_df.empty: test_df.loc[:, 'split'] = 'test'
    
    # Mover archivos
    for df_split in [train_df, validation_df, test_df]:
        if not df_split.empty:
            for _, row in df_split.iterrows():
                origen = os.path.join(ruta_fotos_crudas, row['nombre_archivo'])
                destino = os.path.join(ruta_dataset_final, row['split'], row['etiqueta'], row['nombre_archivo'])
                if os.path.exists(origen):
                    shutil.move(origen, destino)
    
    # Preparar dataframe de retorno
    nuevos_procesados_list = [df for df in [train_df, validation_df, test_df] if not df.empty]
    if not nuevos_procesados_list: return pd.DataFrame()
    
    nuevos_procesados = pd.concat(nuevos_procesados_list).copy()
    nuevos_procesados['fuente'] = nombre_fuente
    return nuevos_procesados[['nombre_archivo', 'etiqueta', 'split', 'fuente']]

# --- INICIO DE LA EJECUCIÓN ---
CLASES = ['reciclable', 'no_reciclable']
SPLITS = ['train', 'validation', 'test']

# Crear estructura de carpetas
for base_path in [RUTA_DATASET_FINAL_PROPIO, RUTA_DATASET_FINAL_EXTERNO]:
    for split in SPLITS:
        for clase in CLASES:
            os.makedirs(os.path.join(base_path, split, clase), exist_ok=True)
print("✅ Estructuras de carpetas verificadas/creadas.")

# Cargar datos
try:
    df_deseado = pd.read_csv(RUTA_CSV_ETIQUETAS)
    df_deseado.dropna(subset=['etiqueta', 'fuente'], inplace=True)
    df_deseado_propio = df_deseado[df_deseado['fuente'] == 'propio'].copy()
    df_deseado_externo = df_deseado[df_deseado['fuente'] == 'externo'].copy()
    print(f"Se encontraron {len(df_deseado_propio)} registros 'propios' y {len(df_deseado_externo)} 'externos' en el CSV.")
    
    try:
        df_actual = pd.read_csv(RUTA_REGISTRO)
        print(f"Se encontró un registro con {len(df_actual)} archivos procesados.")
    except FileNotFoundError:
        df_actual = pd.DataFrame(columns=['nombre_archivo', 'etiqueta', 'split', 'fuente'])
        print("No se encontró registro previo. Se creará uno nuevo.")
    if 'fuente' not in df_actual.columns: df_actual['fuente'] = None
    
    # Ejecutar procesamiento
    nuevos_propios = procesar_fuente(df_deseado_propio, df_actual, RUTA_FOTOS_CRUDAS_PROPIAS, RUTA_DATASET_FINAL_PROPIO, 'propio')
    nuevos_externos = procesar_fuente(df_deseado_externo, df_actual, RUTA_FOTOS_CRUDAS_EXTERNAS, RUTA_DATASET_FINAL_EXTERNO, 'externo')
    
    # Actualizar registro
    df_actual_actualizado = pd.concat([df_actual, nuevos_propios, nuevos_externos], ignore_index=True).drop_duplicates(subset=['nombre_archivo'], keep='last')
    df_actual_actualizado.to_csv(RUTA_REGISTRO, index=False)
    print(f"\n✅ Proceso completado. El registro '{os.path.basename(RUTA_REGISTRO)}' ha sido actualizado.")
    
    # Verificación final
    print("\nConteo total de archivos en los datasets finales:")
    for nombre, ruta in [("Propio", RUTA_DATASET_FINAL_PROPIO), ("Externo", RUTA_DATASET_FINAL_EXTERNO)]:
        print(f"\nDataset {nombre}:")
        for split in SPLITS:
            total_split = sum(len(files) for r, d, files in os.walk(os.path.join(ruta, split)))
            print(f"  - Total en {split}: {total_split} archivos")
    
except (FileNotFoundError, KeyError) as e:
    print(f"❌ ERROR: No se pudo completar el proceso. Revisa que el archivo CSV exista y tenga las columnas correctas ('etiqueta', 'fuente'). Error: {e}")